In [1]:
import pandas as pd
import numpy as np
import random


np.random.seed(42)

# Кол-во пользователей
USER_QUANT = 10000
# Кол-во айтемов
ITEM_QUANT = 100
# Мат ожидание кол-ва успещных взаимодействий
ACTION_QUANT = 10
# Размер одновременно отображаемых значений
WINDOW_SIZE = 6
# Вклад интересов пользователя
INTERSTS_SCORE = 0.4
# Вклад пола пользователя
SEX_TYPE_SCORE = 0.4
# Коэффициент сглаживания скора
POW_VAL = 0.7
# Возможные жанры
GENRES = ["Драма", "Комедия", "Триллер", "Мелодрама", "Фантастика"]
# Количество жанров - интересов пользователя
NUM_INTERESTS_GENRES = 3
# пол пользователя
SEX_TYPES = ["M", "F"]
# действия пользователя
ACTIONS = ['show', 'click']

# создаем список пользователей и айтемов
users = [str(i) for i in range(USER_QUANT)]
items = [str(i) for i in range(ITEM_QUANT)]


users_df = pd.DataFrame({
    'user_id': users,
    'sex_type': [random.choice(SEX_TYPES) for _ in range(USER_QUANT)],
    # Пользователю присваевается NUM_INTERESTS_GENRES скрытых интересов
    "interests": [random.sample(GENRES, k=NUM_INTERESTS_GENRES) for _ in range(USER_QUANT)],
})



items_df = pd.DataFrame({
    'item_id': items,
    'genre': [random.choice(GENRES)  for _ in range(ITEM_QUANT)],
    # Айтем может быть интересен представителем конкретного пола или не быть интересен никому
    "interests_sex": [random.choice(SEX_TYPES + [""])  for _ in range(ITEM_QUANT)],
    "mean_users_score": np.random.uniform(size=ITEM_QUANT) * 5
})


# Создадим датасет качества объектов
item_quality = items_df[['item_id', 'mean_users_score']]
item_quality["item_quality"] = item_quality['mean_users_score']
item_quality["item_quality"] = item_quality["item_quality"] / item_quality["item_quality"].sum()
item_quality_sorted = item_quality.sort_values(by='item_quality', ascending=False).reset_index(drop=True)
# Мы будем считать, что что бы долистать дор обхъекта нужно сдлеать несколко скроллов
item_quality_sorted["window"] = item_quality_sorted.index.values // WINDOW_SIZE

# Рассчитываем датасет с необходимыми скроллами
data_windows = pd.DataFrame(columns=["item_id", "item_showed"])
for i in range(item_quality_sorted["window"].max()):
    items_block = item_quality_sorted[item_quality_sorted["window"] == i][["item_id"]]
    showed_items = item_quality_sorted[item_quality_sorted["window"] <= i][["item_id"]].rename(columns={"item_id": "item_showed"})
    df_merged = pd.merge(items_block, showed_items, how='cross')
    data_windows = data_windows._append(df_merged)

# Оцениваем скоры релевантности
user_probs = pd.merge(users_df, item_quality_sorted, how="cross").merge(items_df, on="item_id")
user_probs["score"] = user_probs.apply(lambda x: x["item_quality"] + INTERSTS_SCORE * int(x["genre"] in x["interests"]) + SEX_TYPE_SCORE * int(x["interests_sex"] == x["sex_type"]), axis=1)
# отнормируем скоры так, что бы в среднем на пользователя приходилось ACTION_QUANT действий
user_probs["score"] = user_probs["score"] / user_probs["score"].mean() * ACTION_QUANT / 100
user_probs["click_flag"] = user_probs["score"] > np.random.uniform(size=user_probs.shape[0])
data_click = user_probs[user_probs["click_flag"] == 1]
data_click = data_click[["user_id", "item_id"]]
data_click["action"] = ACTIONS[1]
# Добавим даты для последовательных моделей
data_click['datetime'] = pd.to_datetime(np.random.rand(data_click.shape[0]) * 10000000000000000) + pd.DateOffset(years=55)

# Добавим какие объектны нужно было пролистнуть пользователю, что бы увидеть объект на которы йон кликнуд
data_show = data_click.merge(data_windows, on='item_id', how='left')[["user_id","item_showed", "action", 'datetime']].rename(columns={"item_showed": "item_id"})
data_show["action"] = ACTIONS[0]

out = data_click._append(data_show, ignore_index=True)

# Сохраняем результат
out.to_csv('data/interactions.csv', index=False)
users_df.drop(columns=["interests"]).to_csv('data/users.csv', index=False)
items_df.drop(columns=["interests_sex"]).to_csv('data/items.csv', index=False)
print(out)
print(items_df)

/var/folders/ld/vywms60d4_7281vl54t27ngwdqgvkl/T/ipykernel_81467/1339224289.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_quality["item_quality"] = item_quality['mean_users_score']
/var/folders/ld/vywms60d4_7281vl54t27ngwdqgvkl/T/ipykernel_81467/1339224289.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_quality["item_quality"] = item_quality["item_quality"] / item_quality["item_quality"].sum()


        user_id item_id action                      datetime
0             0      69  click 2025-01-17 10:04:17.216954590
1             0      53  click 2025-01-11 02:47:00.259085350
2             0      92  click 2025-03-11 04:40:18.978093997
3             0      19  click 2025-02-08 00:59:21.139311191
4             0      64  click 2025-01-15 15:46:51.046877007
...         ...     ...    ...                           ...
5006386    9999       0   show 2025-02-19 15:58:26.873558828
5006387    9999      23   show 2025-02-19 15:58:26.873558828
5006388    9999      78   show 2025-02-19 15:58:26.873558828
5006389    9999      63   show 2025-02-19 15:58:26.873558828
5006390    9999      82   show 2025-02-19 15:58:26.873558828

[5006391 rows x 4 columns]
   item_id       genre interests_sex  mean_users_score
0        0  Фантастика             M          1.872701
1        1     Триллер             F          4.753572
2        2   Мелодрама             F          3.659970
3        3  Фантасти